<a href="https://colab.research.google.com/github/Charee-Villapong/charee-villapong/blob/master/get_tweet_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# 必要なモジュールのimport
import datetime
import pandas as pd
import tweepy

In [39]:

# 各種ツイッターのキーをセット　consumer_key, consumer_secret, access_key, access_secret
consumer_key = "8ljKZRP1ekS9dzkhuaUr6mL8J"
consumer_secret = "fcpAsRKs4wg2jnUc98jJ1YG4kDLcumehO76T2CAQXH74gPeHjs"
access_key = "1289362986210009089-LYBDoTh8ipkpbCnOfy0DpaIfqQM7ZM"
access_secret = "loSkIIcmgZ7YmIx3y4IBqfTYIfN8aYc3Jva41SIFSmflP"

# 認証のためのAPIキーをセット
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth)
#CjtzvLBhCeBmksgJE5B1qdTzo


# API利用制限にかかった場合、解除まで待機する

def main():
    '''
    メインの実行部分
    ツイートデータの取得からデータの出力まで
    '''

    # ツイートデータの取得 日付の指定は 2020-7-30のみでもOK,
    # 日本時間で取得したい場合は2020-7-30_00:00:00_JSTのように指定
    # JSTをつけないと時間がUTCになる UTCは協定世界時間-> JST＋9:00(日本時間よりも9時間進んでいる)
    tweet_data = get_serchtweet('python','2021-01-22,00:00:00_JST','2021-01-29,00:00:00_JST',100,10)
    # ツイートデータを番号順に出力
    output_tweets(tweet_data)
    # ツイートデータをDataframeにする
    df = make_df(tweet_data)
    # ツイートデータのCSVへの出力
    df.to_csv('tweet_data.csv')


# ツイートを収集する関数
def get_serchtweet(s,since,until,count,rlcount):
    '''
    ツイート情報を特定のキーワードで、期間を指定して収集
    取得できるデータは1週間以内のデータだけ
    リツイート数＋いいね数の合計でツイートを絞り込める
    '''

    # 検索キーワードの設定、 リツイートは除く
    search_key = s + "-filter:retweets"
    # ツイートデータ取得部分
    # tweepy.CursorのAPIのキーワードサーチを使用(api.search)
    # qがキーワード, sinceがいつから, untilがいつまで, tweet_modeでつぶやきの省略ありなし, langで言語, .items(数)と書いてツイート数を指定
    tweets = tweepy.Cursor(api.search,q=search_key,since = since ,until = until,tweet_mode = "extended", lang="ja").items(count)
    # ツイートのデータを取り出して、リストにまとめていく部分
    
    # ツイートデータを入れる空のリストを用意
    tweet_data = []
    for tweet in tweets:
        if tweet.retweet_count + tweet.favorite_count >= rlcount:# いいねとリツイートの合計がrlcuont以上の条件
            tweet_data.append([tweet.user.name,tweet.user.screen_name,tweet.retweet_count,tweet.favorite_count,tweet.created_at.strftime("%Y-%m-%d-%H:%M-%S"),tweet.full_text.replace("\n","")])# 空のリストにユーザーネーム、スクリーンネーム、RT数、いいね数、日付などを入れる
    return tweet_data

def output_tweets(tweet_data):
    '''
    ツイートのリストを順番をつけて出力する関数の作成
    '''
    for i in range(len(tweet_data)):
        print(f"({i}番目のつぶやき{tweet_data[i]}")


def make_df(tweet_data):
    '''
    ツイートデータからDataFrameを作成する
    '''

    # データを入れる空のリストを用意(ユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文)
    list_username = []
    list_userid = []
    list_rtcount = []
    list_favorite = []
    list_date = []
    list_text = []

    # ツイートデータからユーザー名、ユーザーid、RT数、いいね数、日付け、ツイート本文のそれぞれをデータごとにまとめたリストを作る
    for i in range(len(tweet_data)):
        list_username.append(tweet_data[i][0])
        list_userid.append(tweet_data[i][1])
        list_rtcount.append(tweet_data[i][2])
        list_favorite.append(tweet_data[i][3])
        list_date.append(tweet_data[i][4])
        list_text.append(tweet_data[i][5])

    # 先ほど作ったデータごとにまとめたリストからDataframeの作成
    df = pd.DataFrame({'ユーザーID':list_userid,
                'ユーザー名':list_username,
                'RT数':list_rtcount,
                'いいね数':list_favorite,
                '日時':list_date,
                '本文':list_text})

    return df
#実行部分
if __name__ == '__main__':
    main()

(0番目のつぶやき['楽(らく)❁', 'GgJywhoOncvrrUh', 1, 11, '2021-01-28-23:53-57', '今日帰ったら作ります。こんな感じの。ゲーム。自作します。Pythonとかで。(完成するかは未定) https://t.co/jZH7LT4KwV']
(1番目のつぶやき['ruby&rails初心者@駆け出しエンジニアと繋がりたい', 'vetlanprogramer', 0, 17, '2021-01-28-23:53-34', 'RubyなんてやるとiOTの仕事に就けないぞ。素直にC#やPythonやっておけ\u3000仕事枠ないぞ\u3000給料安いぞ\u3000大手に就職できないぞ\u3000早く学びなおしたほうがいい 21   #駆け出しエンジニアと繋がりたい']
(2番目のつぶやき['吉田翼＠陽気な陰キャ&WEB制作', 'y_tsubasa_0205', 0, 25, '2021-01-28-23:41-22', '#今日の積み上げ9〜12時     レポート作成13〜15時\u3000Python学習16〜20時    バイト20〜24時    レポート作成レポート終わったら、春休み！！！！春休みって言っても、コーティング&amp;就活&amp;バイトだらけ笑ようは、自分の好きなように時間が使えるのが良き☺️#駆け出しエンジニアと繋がりたい']
(3番目のつぶやき['小野あっと', 'onodesafio', 0, 11, '2021-01-28-23:37-39', 'おはようございます！今日も今日とて前進します🔥#今日の積み上げ ・筋トレ・Python osモジュール・読書(ひろゆきさんの世の中の真実買います😁)']
(4番目のつぶやき['Tommy@セブ島駆け出しブロガー', 'tommy2blog', 0, 27, '2021-01-28-23:32-58', '【ブログ更新】- #今日の積み上げ- #ブログ更新しました- 今日の #python\u3000で #ブログ書け- 今日も #界王拳 で #作業ロケット- もっと #プログラミング初心者 や #駆け出しエンジニアと繋がりたい【python】confluenceを操作する方法【→業務効率UP】 https://t.co/EULKWO7eSS']
(